In [1]:

#r "nuget: xunit, 2.8.1"
using System.Threading;
using Xunit;

class Message
{
    static int messageHandled = 0;
    int id;
    public Message()
    {
        id = Thread.CurrentThread.ManagedThreadId;
    }

    public void Handle()
    {
        Interlocked.Increment(ref messageHandled);
        Assert.NotEqual(id, Thread.CurrentThread.ManagedThreadId);
    }

    public static void Check()
    {
        Assert.Equal(200, messageHandled);
    }
}



Installed Packages xunit, 2.8.1

In [2]:
using System.Collections.Concurrent;
using System.Threading;
using System.Threading.Tasks;

var ocher1 = new BlockingCollection<Message>(2); //очередь первого потока
var ocher2 = new BlockingCollection<Message>(2); //очередь второго потока


// Создаем два потока
var potoc1 = new Thread(() =>
{
   
    for (int i = 0; i < 100; i++)
    {
        // Читаем сообщение из очереди первого потока
        var message = ocher1.Take();
        // Обрабатываем сообщение
        message.Handle();
       
       
        {
            var newMessage = new Message();
            ocher2.Add(newMessage);
        }
    }
    
});
var potoc2 = new Thread(() =>
{ int handled2=0;
    for (int i = 0; i < 100; i++)
    {
        // Читаем сообщение из очереди второго потока
        var message = ocher2.Take();
        // Обрабатываем сообщение
        message.Handle();
        handled2++;
        // Создаем новое сообщение и записываем его в очередь первого потока
        if(handled2!=100)
        {
            var newMessage = new Message();
            ocher1.Add(newMessage);
        }
       
        
    }
    
});
// Создаем первое сообщение и записываем его в очередь первого потока
var firstMessage = new Message();
ocher1.Add(firstMessage);

// Запускаем потоки
potoc1.Start();
potoc2.Start();

// Дожидаемся завершения работы потоков
Parallel.Invoke(()=>
{
    potoc1.Join();
},
()=>
{
    potoc2.Join();
}
);


In [3]:
Assert.Equal(0, ocher1.Count);
Assert.Equal(0, ocher2.Count);
Message.Check();